Nom Etudiant 1: Shiyao CHEN
<br>
Nom Etudiant 2: Ménalie SELLAHENNEDI

# TME 3.2: Projet Detection de motifs


## Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons d'abord implémenter l'algorithm random Projections.


1\.  Nous allons réutiliser les fonctions du precedent pour générer t séquences artificielles de taille n, et implanter dans chaque séquence un motif de taille k à des positions aléatoires avec v substitutions choisies aléatoirement. Nous allons faire varier le motifs dans 50% de cas.

In [12]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence
f=0.5

In [13]:
def insertMotif(sequence, motif, position):
    return sequence[:position] + motif + sequence[position:]

def generateRandomSequence(n, upper=True):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront en majuscule, False minuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""
    for i in range (n):
        s=random.choice(nuc)
        if(upper):
            sequence=sequence+(s.upper())
        else:
            sequence=sequence+(s.lower())
    
    return sequence

def modifierMotif(motif, nbpos,  upper=True):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront majuscule, False minuscule
    sortie motifM: motif modifié
    """
    motifM = list(motif)
    #print(motif)
    for i in range(0,nbpos):
        t=random.randint(0,len(motifM)-1)
        while (i>0 and t==prec): #pour verifier qu'on ne s'occupe pas d'une même position 2 fois de suite
            t=random.randint(0,len(motifM)-1)
            
        s=random.choice(nuc)
        #print(s==motifM[t].upper())
        while (s==motifM[t].upper()): #pour verifier qu'on ne substitue pas par la même lettre
            s=random.choice(nuc)
        
        if (upper==True):
            motifM=motifM[:t]+[s.upper()]+motifM[t+1:]
        else:
            motifM=motifM[:t]+[s.lower()]+motifM[t+1:]
        prec=t
    
    #print(motifM)
    return "".join(motifM)

#tester la fonction modifierMotif
#print (modifierMotif("acg", 2))


def implantMotifVar(k, v, t, n, f):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    entrée f : frequence de variation du motif.
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    sequences = []
    for i in range(0,t):
        if(random.uniform(0.0, 1.0)<=f):
            pos=random.randint(0,n)
            sequences.append(insertMotif(generateRandomSequence(n-k,False),modifierMotif(generateRandomSequence(k,False),v),pos))
        else:
            sequences.append(generateRandomSequence(n,False))
    return sequences


adn = implantMotifVar(k, v, t, n, f)
print (adn)

adn  = [s.upper() for s in adn]
print (adn)

['ttggcccgtg', 'tcagcctgTt', 'cttcactaGa']
['TTGGCCCGTG', 'TCAGCCTGTT', 'CTTCACTAGA']


2\. Nous allons implémenter l'algorithme ``randomProjection``. D'abord, faites la fonction `getRandomFixePositions` pour générer une projection de p à k, voir un exemple dans les slides de cours. Faire aussi la fonction `generateKey` qui extrait les caractères du motif puis génère une cle qui représente la projection.

In [3]:
def is_in(x, L):
    for el in L:
        if (x==el):
            return True
    return False

def getRandomFixePositions(p, k):
    """
    Genere une projection de p vers k
    entrée p: nombre de positions choisi pour la projection 
    entrée k: nombre de positions du motif original
    sortie projection: liste de positions choisi aléatoirement
    """
    projection = []
    count = 0
    prec=[]
    for i in range(0,p):
        pos=random.randint(0,k-1)
        while(is_in(pos,prec)):
            pos=random.randint(0,k-1)
        projection.append(pos)
        prec.append(pos)
    return projection

lR = getRandomFixePositions(4, 7)
lR.sort()
print(lR)

def generateKey(projection, motif):
    """
    extrait les caractères du motif et génère la cle de la projection
    entrée projection : liste de positions qui represent la projection
    entrée motif : motif de taille k
    sortie cle : cle de la projection
    """
    cles = ""
    for i in projection:
        cles+=motif[i]
    return cles

generateKey(lR, "01234567")

[0, 1, 3, 5]


'0135'

3\. Implémenter l'algorithme ``randomProjection``. Bonnus : Pour ameliorer la performance vous pouvez abandonner les motifs de taille k peu complexes.

In [4]:
def removeLowComplexe(motifs,k):
    motifsClean = []
    supp=False
    for key in motifs:
        for i in range(len(key)):
            cpt=key.count(key[i])
            if (cpt>=k):
                supp=True
            if ((i+3)<len(key)):
                if key[i]==key[i+2] and key[i+1]==key[i+3]:
                    supp=True
        if (supp==False):
            motifsClean.append(key)
        supp=False
    return motifsClean



def randomProjection(k, p, sequences):
    """
    Implémente l'algorithme randomProjection
    entrée k : taille du motif
    entrée p : nombre de positions de la projection 
    entrée sequences : matrice de dimension txn contenant les séquences 
    sortie motifs : dictionaire, cle = projection, valeur= frequence
    sortie motifsSeq:  dictionaire, cle = projection, valeur= original motif
    """
    motifs  = {}; motifsSeq  = {}
    projection=getRandomFixePositions(p, k)
    for seq in sequences:
        i=0
        all_motifs=[]
        for i in range(len(seq)-k+1):
            motif=seq[i:i+k]
            all_motifs.append(motif)
            all_motifs=removeLowComplexe(all_motifs,k) # abandonner les motifs de taille k peu complexes
            if is_in(motif,all_motifs):
                cle=generateKey(projection,motif)
                if (is_in(cle,motifs.keys())):
                    motifsSeq[cle].append(motif)
                    motifs[cle]=motifs[cle]+1
                else:
                    motifsSeq[cle]=[motif]
                    motifs[cle]=1
    return motifs, motifsSeq

#motifsSort = sorted(motifs, reverse=True, key=motifs.get)

adnTest = ['TTAACGGCAC', 'GCTCACGCCA', 'TACCGGCCGT']
motifsProj, motifsSeq = randomProjection(7, 4, adnTest)
print (motifsProj)
print (motifsSeq)

#motifsProj => {'TACG': 1, 'TCGC': 3, 'AGGA': 1, 'AGCC': 1, 'GCAG': 1, 'CACC': 1, 'CGCA': 1, 'AGGC': 1, 'CGCG': 1, 'CCCT': 1}
#motifsSeq => {'TACG': ['TTAACGG'], 'TCGC': ['TAACGGC', 'TCACGCC', 'TACCGGC'], 'AGGA': ['AACGGCA'], 'AGCC': ['ACGGCAC'], 'GCAG': ['GCTCACG'], 'CACC': ['CTCACGC'], 'CGCA': ['CACGCCA'], 'AGGC': ['ACCGGCC'], 'CGCG': ['CCGGCCG'], 'CCCT': ['CGGCCGT']}

{'ACGT': 2, 'CGGA': 2, 'GGCA': 1, 'GCAC': 1, 'CACC': 1, 'CGCC': 1, 'GCCA': 1, 'GGCC': 1, 'GCCC': 1, 'CCGG': 1}
{'ACGT': ['TTAACGG', 'CTCACGC'], 'CGGA': ['TAACGGC', 'TACCGGC'], 'GGCA': ['AACGGCA'], 'GCAC': ['ACGGCAC'], 'CACC': ['GCTCACG'], 'CGCC': ['TCACGCC'], 'GCCA': ['CACGCCA'], 'GGCC': ['ACCGGCC'], 'GCCC': ['CCGGCCG'], 'CCGG': ['CGGCCGT']}


4\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour augmenter les chances de le trouver. 

reponse: Etant donnée qu'il n'y a qu'une itération de l'algorithme même si on execute plusieurs fois, les buckets se réinitialise à chaque fois et la probabilité qu'un bucket se remplisse beaucoup est assez faible d'où l'incapacité à trouver le meilleur score d'un motif et donc le motif implanté. Mais on peut estimer qu'il se trouve dans ['TAACGGC', 'TACCGGC'] comme c'est le bucket au score le plus élevé.

5\. Implémenter la version itérative de l’algorithme ``randomProjection``. 

In [5]:

#Construire matrice de fréquence
def profile(motifs, k, nuc):
    """
    Construire une matrice de fréquence de dimension k x |nuc|
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie MF: matrice de fréquence
    """
    q = len(nuc)
    PWM = np.zeros((q, k))
    i=0
    j=0
    for i in range (0,k):
        for j in range (len(motifs)):  # 1er ligne =A  2eme ligne =C   3eme ligne =G   4eme ligne =T
            lettre=motifs[j][i]
            if lettre=="A":
                PWM[0][i]+=1
            if lettre=="C":
                PWM[1][i]+=1
            if lettre=="G":
                PWM[2][i]+=1
            if lettre=="T":
                PWM[3][i]+=1
    
    return PWM

def getScore(MF, k):
    """
    Renvoie le score de MF, la somme des max de chaque colonne
    entrée MF: matrice de fréquence
    entrée k: taille du motif
    sortie sc: score
    """
    sc = 0
    i=0
    j=0
    for i in range(len(MF[0])):
        max_col=0
        for j in range (len(MF)):
            s=MF[j][i]
            if(max_col<s):
                max_col=s
        #print(max_col)
        sc=sc+max_col
    
    return sc

In [6]:

    
def randomProjIt(sequences, k, v, nuc, It):
    """
    Implémente l'algorithme randomProjection version iteractive
    entrée sequences : matrice de dimension txn contenant les séquences 
    entrée k : nombre de positions du motif
    entrée v : nombre de positions de la projection 
    entrée nuc : alphabet
    entrée It: nombre d'iterations
    sortie score : meilleur score
    sortie motifs :  liste de motifs associés au meilleur score
    """
    score=0
    motifs = []; scores = 0
    for i in range (0,It):
        algo=randomProjection(k, v, sequences)
        #print(i,algo)
        #all_motifs=allMotifsFromDict(algo[1]) #on recupère l'ensemble des motifs
        for val in algo[1].values():#on parcourt les buckets de motifs
            #print(algo[1])
            #print(val)
            current_score=getScore(profile(val,k,nuc),k) #plus cette fonction donne une valeur grande plus le bucket dans lequel se trouve le motif est grand
            #print(current_score)
            if score<current_score:
                score=current_score
                motifs=val
    return score, motifs

score, seqsMotif = randomProjIt(adnTest, 7, 4, nuc, 100)
print(adnTest)
print (score, seqsMotif)


['TTAACGGCAC', 'GCTCACGCCA', 'TACCGGCCGT']
18.0 ['TAACGGC', 'TCACGCC', 'TACCGGC']


6\. Tester l'algorithme  ``randomProjection`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats. Puis générér le LOGO du motif trouvé.

In [7]:
def reverseComplement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in reversed(seq)])

def complement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in seq])

def printTopFMotifsFreq(motifs, m, rev=False):
    motifsRet = {}
    motifsSort = sorted(motifs, reverse=True, key=motifs.get)
    i = 0
    while (i < m):
        motifPrint = motifsSort[i]
        print (motifsSort[i])
        if rev:
            motifPrint = reverseComplement(motifsSort[i])
        print (i, motifPrint, "-", motifs[motifsSort[i]])
        i = i + 1

motifs, motifsSeq = randomProjection(7, 4, adnTest)
printTopFMotifsFreq(motifs, 10)
#print (motifsSeq['CATA'])

CTGG
0 CTGG - 2
GTGC
1 GTGC - 1
AACG
2 AACG - 1
CAAC
3 CAAC - 1
GGCA
4 GGCA - 1
CCGC
5 CCGC - 1
CTCG
6 CTCG - 1
ACCC
7 ACCC - 1
CACG
8 CACG - 1
GCCC
9 GCCC - 1


In [8]:
k=7; p=4; n=80

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    #sequence = [sequenceStr]
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "Sequence_by_Peaks_8.fasta"

sequencesChip   = readFasta(genome, n)
t = len(sequencesChip)
print (sequencesChip[8], t, n, k)
sequencesChip = [x for x in sequencesChip if len(x)==n] # on a enlevé les séquences dont la taille n'est pas égale à n
revSequences = [reverseComplement(m) for m in sequencesChip]
sequences = sequencesChip + revSequences
score, seqsMotif = randomProjIt(sequences, k, p, nuc, 100)
print("Meilleur score: ",score)
print(seqsMotif)

GTGTGAGTCTCGTTGTACGTCCGAGTTCTCGGTTTTTTACGAAACAAATCCCCTTCGGCTGCGCGGTTGCCTCGCGGTCC 400 80 7
Meilleur score:  2073.0
['AATCTAA', 'AATTTAA', 'AAACAAA', 'AATGGAA', 'AAATTAA', 'AAGATAA', 'AAGATAA', 'AACTCAA', 'AAGATAA', 'AATTTAA', 'AAGATAA', 'AAAGAAA', 'AAATGAA', 'AAATTAA', 'AATTAAA', 'AAATTAA', 'AATTAAA', 'AATGGAA', 'AAAGAAA', 'AAAGAAA', 'AACAGAA', 'AAAGAAA', 'AAACTAA', 'AACTAAA', 'AAAGAAA', 'AATTAAA', 'AACTTAA', 'AACCTAA', 'AATACAA', 'AATTAAA', 'AAAGAAA', 'AATCCAA', 'AAGATAA', 'AAGTCAA', 'AACTCAA', 'AAACAAA', 'AACGTAA', 'AAAGCAA', 'AAGCAAA', 'AAAGAAA', 'AATACAA', 'AAGTTAA', 'AAATAAA', 'AAGGCAA', 'AAGGCAA', 'AATGAAA', 'AAAGGAA', 'AAATCAA', 'AATCAAA', 'AATCAAA', 'AAAGAAA', 'AAACTAA', 'AACTAAA', 'AATAGAA', 'AAGTAAA', 'AAAGTAA', 'AAGTAAA', 'AATGGAA', 'AATACAA', 'AAATCAA', 'AATCAAA', 'AATAGAA', 'AAACGAA', 'AAATAAA', 'AAGTAAA', 'AATTCAA', 'AATTAAA', 'AAGGGAA', 'AATGCAA', 'AATACAA', 'AAACTAA', 'AATTAAA', 'AAAGTAA', 'AAGTAAA', 'AATTCAA', 'AATACAA', 'AATTCAA', 'AATACAA', 'AAATTAA', 'AATTAAA', 'AATG